# EM Algorithm v1.2
This is the Revise version of EMCore written by Prof. Chang.
This algorithm is divided into the following steps:

## Preprocessing
    1. Reading CSV File (csv)
    2. Creating OneHot Lot Encoding of Machines [1]_
    3. Preprocessing: Calculating LOT Information
    4. Preprocessing: Calculating Machine Information

the initial yieldrate is calculated using Least Square and L-BFGS-B [2]_, and is
included id step 4 above.

## EM Algorithm
    5. Running EM Algorithm

## Report
    6. Saving reports to CSV

## Optimization
some of the code is optimize based on Python Speed Performance Tips recommended
by python.org [3]. In this code, it could be seen commented as
``Optimizing:...``. One of them are avoiding dots (e.g. avoid multiple calls
of ``df.my_column``)


## References
.. [1] Jezrael. 2019. Answer on "Create dummy variable of multiple columns with
   python". One hot encoding with pandas "get_dummies" and different column and
   same values is seen as one values. `link <https://stackoverflow.com/
   questions/55182909/create-dummy-variable-of-multiple-columns-with-python>`_

.. [2] The SciPy community. 2019. Limited-memory BFGS B. see: `scipy.optimize.
   fmin_l_bfgs_b <https://docs.scipy.org/doc/scipy/reference/generated/
   scipy.optimize.fmin_l_bfgs_b.html>`_

.. [3] Python.org. 2016. Python Speed Performance Tips. `link <https://wiki.
   python.org/moin/PythonSpeed/PerformanceTips>`_

In [37]:
!pip install pandas==1.5.3



---


# Initializing Google Colab

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Please modify the path below to match your folder path
change "ColabNotebook/TH" with your folder path in your google drive

In [39]:
cd '/content/gdrive/MyDrive/Colab Notebooks'

/content/gdrive/MyDrive/Colab Notebooks


## checking required file

In [40]:
!ls custom_utility.py
!ls test_case_1a.csv
!ls real_world_data.csv

custom_utility.py
test_case_1a.csv
real_world_data.csv


In [41]:
!cat custom_utility.py

# -*- coding: utf-8 -*-
"""
Created on Sat Jul 13 11:50:43 2019

@author: Faster

Importing:
    first, this file should be in the same folder as the importer, then

    >>> import custom_utility as cu

    or using Python's Relative imports, Every leading dot is another higher
    level in the hierarchy beginning with the current directory.

    >>> import ...app.folder.file as cu

Features:
    show_progress:
        Print the current progress in percentage
    check_time:
        Calculate and print time passes from ``time_start`` to ``time_end``.
"""

import sys

def show_progress(current_num, max_num, step_print=5, messages='',
                  force_show=False):
    """
    Print the current progress in percentage

    Parameter:
    ----------
    current_num, max_num: int
        The current number and the maximum number of the progress or loop
        (if used)
    step_print: int (optional)
        (unit is in percent) How often the message should be printed, the
        def

## Importing modules

In [42]:
# import sys
# sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/custom_utility.py')

In [43]:
import time
import numpy as np
import pandas as pd
import scipy.optimize
import warnings

import custom_utility as cu
from os import listdir
from os.path import isfile, join

# Code Configurations

In [44]:
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# using 'raw_real_data.csv' or 'test_case_1a.csv'
# RAWDATA_CSV_PATH_FILENAME = 'test_case_1a.csv'
# real_world_data
RAWDATA_CSV_PATH_FILENAME = 'real_world_data.csv'
# REPORT_PATH_FILENAME = "yield_rate_report_v1.2.csv"
# REPORT_PATH_FILENAME = "report_test_case_1a.csv"
REPORT_PATH_FILENAME = "report_real_world_data.csv"
MAX_EMA_ITERATIONS = 25
EPSILON = 1.e-99 # a small positive infinitesimal quantity
OLD_COLUMNS = {"lot name" : 'Lot',
               "process prefix" : 'Process',
               "process suffix" : '_MachineNo',
               "lot step" : 'Final',
               "processed" : 'CurrentTotalNum',
               "bad pieces" : 'BadNum',
               "total lot pieces" : 'TotalNum'
              }
NEW_COLUMNS = {"lot name" : 'lot_name',
               "process prefix" : 'lot_machine_step',
               "lot step" : 'lot_step',
               "processed" : 'processed',
               "bad pieces" : 'bad_pieces',
              }

# Preprocessing

## Read CSV file

In [45]:
def read_csv_files(filename):

    """
    read the preprocessed data with given path and filename:

    Parameter:
    ----------
    filename : str
        given path and filename such as ``"folder/yourfilename.csv"``

    Return:
    ----------
    raw_csv : DataFrame or TextParser (2D array)
        full data from your csv.

    """
    #read the chinese character with encoding big5
    #raw_csv = pd.read_csv(filename, encoding='big5', low_memory=False)
    raw_csv = pd.read_csv(filename, encoding='big5',
                          iterator=True, chunksize=10000)
    raw_csv = pd.concat(raw_csv, ignore_index=True)

    # renaming columns if lot name columns appear as old name
    if OLD_COLUMNS["lot name"] in raw_csv.columns:
        # make sure the conversion of process column
        _col_1 = len(set(raw_csv.CurrentTotalNum))
        _col_2 = len(set(raw_csv.TotalNum))
        process_col = 'CurrentTotalNum'
        if _col_1 < _col_2:
            process_col = 'TotalNum'


        #rename default name
        raw_csv.rename(columns={
            OLD_COLUMNS["lot name"]: NEW_COLUMNS["lot name"],
            OLD_COLUMNS["lot step"]: NEW_COLUMNS["lot step"],
            process_col: NEW_COLUMNS["processed"],
            OLD_COLUMNS["bad pieces"]: NEW_COLUMNS["bad pieces"]
        }, inplace=True)

        # remove old process column suffix
        raw_csv.columns = [col.replace(OLD_COLUMNS["process suffix"], '')
                           for col in raw_csv.columns]
        # rename all prefix of machine process column
        raw_csv.columns = [col.replace(OLD_COLUMNS["process prefix"],
                                       NEW_COLUMNS["process prefix"])
                           for col in raw_csv.columns]

    # fix unconsistent bad pieces:
    # when processed pieces is reduced in next row [shift(-1)],
    # then re-calculate all bad pieces in current row
    # except for the last step (-1)
    raw_csv['all_bad_pieces'] = raw_csv.bad_pieces
    raw_csv.loc[raw_csv['lot_step'] != -1, 'all_bad_pieces'] \
        =  raw_csv.processed - raw_csv.processed.shift(-1)
    return raw_csv

## One Hot Encoding

In [46]:
def get_onehot_machine(source_data):
    """
    preprocess the machine name in each step with one hot encoding, then return
    hotencoding including machine name and updated source data.

    Parameter:
    ----------
    source_data : DataFrame (2D array)
        insert full source data from your csv.

    Return:
    ----------
    onehot_last_step : DataFrame (2D array)
        an each machine step feature in one hot encoding format, with
        machine name as its columns name
    machine_name : DataFrame (2D array)
        a machine information dataframe which only consist of machine name.
    source_data : DataFrame (2D array)
        full data source from your csv, but added a new column called
        ``lot_step_machines`` which consist of machines name that used in
        each LOT's step

    """
    #select the machine step columns (backslash to break, avoid long lines)
    #select column "ProcessXXX_MachineNo"
    machine_step_data = source_data.loc[:, source_data.columns.str.                 \
                                        startswith("lot_machine_step")]

    # one hot encoding with pandas "get_dummies" and
    # different column and same values is seen as one values
    onehot_machine_step = pd.get_dummies(machine_step_data,
                                         prefix='',
                                         prefix_sep=''
                                        ).astype(np.int8).max(level=0 ,axis=1)

    # get last step index of each LOT (from raw data)
    last_step_index = source_data.lot_step[source_data.lot_step == -1]  \
                                 .index                                 \
                                 .tolist()

    # filter the index based on last step raw data for one hot encoding
    onehot_last_step = onehot_machine_step.iloc[last_step_index]
    # machine name as dataframe
    machine_name = pd.DataFrame({'machine_name':list(onehot_last_step)})
    #machine_name['index'] = machine_name.index

    #get the machine used for every row, based on machine config
    lot_step_machines = machine_step_data.copy()

    # change dataframe to tuple in 4 steps
    # Step 1: get non NaN/Null value each row using "stack" data structure
    lot_step_machines = lot_step_machines.stack()
    # Step 2: group by dataframe row / index value (level=0)
    lot_step_machines = lot_step_machines.groupby(level=0)
    # Step 3: change to list for each row using apply depends on group by
    lot_step_machines = lot_step_machines.apply(list)
    # Step 4: change to tuple of list
    lot_step_machines = tuple(lot_step_machines) # change to tuple of list

    # modify the raw data to add new column
    source_data['lot_step_machines'] = lot_step_machines

    return onehot_last_step, machine_name, source_data

## Analyzing Lot/Batch Statistics

In [47]:
def get_lot_information(source_data):
    """
    read the statistic of each lot in given source dataframe.

    Parameter:
    ----------
    source_data : DataFrame (2D array)
        insert full data from your csv.

    Return:
    ----------
    lot_info : DataFrame or TextParser (2D array)
        A statistical dataframe consist of 'lot_name', 'machine_used',
        'lot_total_pieces', 'lot_good_pieces', 'lot_yield_rate',
        and 'ln_lot_yield_rate' for each LOT.

    """
    # function to avoid ZeroDivisionError, and replace result to near zero
    def safe_division(x, y):
        if x == 0 or y == 0:
            return EPSILON
        return x/y

    # initializing dataframe
    _columns = ['lot_name', 'machine_used', 'lot_total_pieces',
                'lot_good_pieces', 'lot_bad_pieces', 'lot_yield_rate',
                'ln_lot_yield_rate']
    lot_info = pd.DataFrame(data=None, columns=_columns)

    # groping the raw data based on LOT Name
    lot_group = source_data.groupby('lot_name')

    # calculating all columns value in lot_info for each row / each LOT
    for _lot_name, _lot_data in lot_group:
        # get row where lot_step = -1
        last_step = _lot_data.loc[_lot_data.lot_step == -1]

        # fix unconsistent bad pieces:
        # find delta between all bad pieces and micro crack bad pieces
        lot_all_defect = _lot_data.all_bad_pieces.sum()
        lot_all_microcrack = _lot_data.bad_pieces.sum()
        lot_badpieces_delta = lot_all_defect - lot_all_microcrack

        # get data for lot_info
        # use .item() to get the value of groupby cells -- deprecated, use iat[0]
        machine_used = last_step.lot_step_machines.iat[0] #.item()
        # syncronized with only micro crack defect
        lot_total_pieces = _lot_data.processed.max() - lot_badpieces_delta
        lot_good_pieces = last_step.processed.iat[0] - last_step.bad_pieces.iat[0]
        lot_bad_pieces = lot_total_pieces - lot_good_pieces

        # use function to safely process the division
        lot_yield_rate = safe_division(lot_good_pieces, lot_total_pieces)
        ln_lot_yield_rate = np.log(lot_yield_rate)

        #append to the list based on _columns name
        current_lot_info = [_lot_name, machine_used, lot_total_pieces,
                            lot_good_pieces, lot_bad_pieces,  lot_yield_rate,
                            ln_lot_yield_rate]

        #append to statistic dataframe
        lot_info.loc[len(lot_info)] = current_lot_info

    return lot_info

## Analyzing Machines Statistics

In [48]:
def get_machine_information(onehot_lot_data, machine_info, lot_info):
    """
    calculate the statistic of each machine from several related sources.

    Parameter:
    ----------
    onehot_lot_data : DataFrame (2D array)
        Dataframe consist of each machine step feature in one hot encoding
        format, with machine name as its columns name
    machine_info : DataFrame (2D array)
        A machine information dataframe which only consist of machine name.
    lot_info : DataFrame (2D array)
        An information dataframe for each LOT.

    Return:
    ----------
    machine_info : DataFrame or TextParser (2D array)
        A machine information dataframe consist of 'machine_name',
        'good_produced', 'init_yieldrate', and 'em_yieldrate' for each machine.

    """
    # Step 1: Calculating Good Pieces Produced for each machine
    # initializing dataframe
    machine_info['good_produced'] = 0

    # for faster indexing, first, change dataframe to dictionary
    # dict(zip(keys, values)) --> to change 2 lists to single dictionary
    machine_info = dict(zip(machine_info.machine_name,
                            machine_info.good_produced))

    # calculate good produced for each machine based on LOT's machine usage
    for _, lot_step in lot_info.iterrows():

        # Optimizing: caching variable for faster get value
        machine_used = lot_step.machine_used
        good_pieces = lot_step.lot_good_pieces

        for machine in machine_used:
            # add or sum good pieces to previous value
            machine_info[machine] += good_pieces

    #change back to dataframe
    machine_info = pd.DataFrame(list(machine_info.items()),
                                columns=['machine_name', 'good_produced'])

    # Step 2: Calculating initial yield rate for each machine
    machine_info['init_yieldrate'] = init_yieldrate_est(onehot_lot_data, lot_info)

    return machine_info

## Calculating Initial Yield Rate

In [49]:
def init_yieldrate_est(onehot_data, lot_info):
    """
    calculating each machine's initial yieldrate using Least Square and
    L-BFGS-B (Limited-memory BFGS).
    see: `scipy.optimize.fmin_l_bfgs_b <https://docs.scipy.org/doc/scipy/
    reference/generated/scipy.optimize.fmin_l_bfgs_b.html>`_

    Parameter:
    ----------
    onehot_lot_data : dataframe (2D array)
        insert the onehot encoding data of the LOT's Last Step
    lot_info : dataframe (2D array)
        insert a dataframe which consist of statistic details of each lot

    Return:
    ----------
    est_yieldrate : list or 1D array
        the yield rate probability using least square algorithm, and minimized
        by a function of L-BFGS-B algorithm.

    """

    #initializing yield rate estimation with all zeros
    max_machine_step = len(onehot_data.columns)
    log_est_yieldrate = np.zeros((max_machine_step,))

    #lot onehot where step == -1
    onehot_data = onehot_data.values #convert to array --> as X on formula

    #get initial yield rate as array
    lot_info = lot_info["ln_lot_yield_rate"].values
    #reshape array with any rows but with 1 column
    lot_yieldrate = lot_info.reshape(-1, 1)  #--> as Y on formula

    #calculate dot product for config and yield rate
    config_dot_config = onehot_data.T.dot(onehot_data)  #--> X^2
    config_dot_yield = onehot_data.T.dot(lot_yieldrate) #--> X.Y

    # least Square
    def perform_least_square(params, cfg, i_yr, c_dot_c, c_dot_y):
        """
        using Limited-memory BFGS
        see: `scipy.optimize.fmin_l_bfgs_b <https://docs.scipy.org/doc/scipy/
        reference/generated/scipy.optimize.fmin_l_bfgs_b.html>`_
        """
        yieldrate = (c_dot_c.dot(params.reshape(-1, 1)) - c_dot_y) / 2
        val = cfg.dot(params.reshape(-1, 1)) - i_yr
        val *= val
        return val.sum()/2, yieldrate

    #Minimize a function using the L-BFGS-B algorithm.
    log_est_yieldrate = scipy.optimize.fmin_l_bfgs_b        \
                        (perform_least_square,
                         log_est_yieldrate,
                         None, (onehot_data,
                                lot_yieldrate,
                                config_dot_config,
                                config_dot_yield
                               ),
                         False,
                         [(None, 0) for _ in range(max_machine_step)])

    #restore from np.log in initial yield rate (remove min and info)
    est_yieldrate = np.exp(log_est_yieldrate[0])

    return est_yieldrate

# EM Algorithm

## E-Step

In [50]:
def e_step(source_data, machine_info):
    """
    The E-STEP of Expectation–Maximization (EM) algorithm.
    This step is used to calculate the hidden variable or Expectation of
    bad pieces in each machine, hence can provide the estimation of the number
    of bad pieces and the number of good pieces produced by each machine.

    Parameter:
    ----------
    source_data : dataframe (2D array)
        full data from your csv, including the 'lot_step_machines' column.
    machine_info : dataframe (2D array)
        A machine information dataframe for each machine.

    Return:
    ----------
    machine_info : dataframe (2D array)
        A machine information dataframe for each machine that already updated
        with adding some columns including 'exp_num_bad_pieces',
        'exp_pieces_going_bad', and 'exp_num_good_pieces'.

    """

    #initializing or reset the curent value to recalculate later
    machine_info['exp_num_bad_pieces'] = 0      # reset
    machine_info['exp_pieces_going_bad'] = 0    # reset
    dict_exp_bad = dict(zip(machine_info.machine_name,
                            machine_info.exp_num_bad_pieces))
    dict_going_bad = dict(zip(machine_info.machine_name,
                              machine_info.exp_pieces_going_bad))

    # creating dictionary for faster lookup in dataframe
    # dict(zip(keys, values)) --> to change 2 lists to single dictionary
    cur_yieldrate = dict(zip(machine_info.machine_name,
                             machine_info.em_yieldrate))

    cur_log_yieldrate = dict(zip(machine_info.machine_name,
                                 machine_info.ln_yieldrate))

    # Calculation is going through all LOT's steps
    for _, lot_step in source_data.iterrows():

        # Optimizing: caching bad_pieces (for faster get value)
        obs_bad_pieces = lot_step.bad_pieces

        # E-Step: Ignore Zero bad Pieces
        if obs_bad_pieces > 0:
            # Optimizing: caching lot_step_machines (for faster get value)
            lot_step_machines = lot_step.lot_step_machines

            ### BEGIN Calculate Number of Expected Bad Pieces ###
            exp_bad_pieces = []

            for step_number, machine in enumerate(lot_step_machines):

                # Calculating E-Step point B1 (see design layout slide 42)
                bad_estimation_b1 = 1 - cur_yieldrate[machine]

                # Calculating E-Step point B2 (see design layout slide 42)
                # callculating in Log version
                bad_estimation_b2 = 0
                previous_machines = lot_step_machines[0:step_number]
                for _machine in previous_machines:
                    bad_estimation_b2 += cur_log_yieldrate[_machine]

                # leaving log version
                bad_estimation_b2 = np.exp(bad_estimation_b2)

                # E-Step: B1 * B2
                # + EPSILON is to avoid true_divide (sum value is zero)
                bad_estimation = bad_estimation_b1 * bad_estimation_b2 + EPSILON
                exp_bad_pieces.append(bad_estimation)


            #calculating % bad pieces for each machine in each LOT row
            exp_percent_bad_pieces = (exp_bad_pieces / sum(exp_bad_pieces))

            #calculating number of bad pieces for each machine in each LOT row
            exp_num_bad_pieces = (obs_bad_pieces * exp_percent_bad_pieces)

            ### END OF Calculate Percentage of Number Bad Pieces ###

            ### BEGIN Calculate Expected Pieces Going Bad ###
            # start with all zero
            exp_pieces_going_bad = np.zeros((len(exp_num_bad_pieces),))

            # reversed loop, calculate from the last index
            # https://stackoverflow.com/questions/529424/
            # traverse-a-list-in-reverse-order-in-python
            for step_number, exp_bad_pieces         \
                        in reversed(list(enumerate(exp_num_bad_pieces))):

                #stop before index = 0
                if step_number > 0:
                    exp_pieces_going_bad[step_number-1]         \
                            = exp_bad_pieces                    \
                              + exp_pieces_going_bad[step_number]

            ### END OF Calculate Expected Pieces Going Bad ###

            # merged bad and going_bad pieces to the appropriate machine
            for step_number, machine in enumerate(lot_step_machines):
                dict_exp_bad[machine] += exp_num_bad_pieces[step_number]
                dict_going_bad[machine] += exp_pieces_going_bad[step_number]

    # after LOT's steps iteration
    # maerge dictionaries to main machine information dataframe
    machine_info['exp_num_bad_pieces'] = machine_info['machine_name'].map(dict_exp_bad)
    machine_info['exp_pieces_going_bad'] = machine_info['machine_name'].map(dict_going_bad)

    # calculating Number of Expected Good Pieces for each machine
    machine_info['exp_num_good_pieces'] = machine_info['exp_pieces_going_bad'] \
                                          + machine_info['good_produced']

    return machine_info

## M-Step

In [51]:
def m_step(machine_info):
    """
    The M-STEP of Expectation–Maximization (EM) algorithm.
    This step is used to RE-calculate the yield rate (parameter) of each machine
    based on the estimated number of bad pieces and the estimated number of
    good pieces calculated in E-STEP of EM Algorithm.

    Parameter:
    ----------
    machine_info : dataframe (2D array)
        A machine information dataframe for each machine that already updated
        with adding some columns including 'exp_num_bad_pieces' and
        'exp_num_good_pieces'.

    Return:
    ----------
    machine_info : dataframe (2D array)
        An updated machine information dataframe for each machine which the
        'em_yieldrate' column is already recalculate based on the E-STEP
        estimation.

    """
    machine_info['sum_exp'] = machine_info['exp_num_bad_pieces']            \
                              + machine_info['exp_num_good_pieces']

    # RE-calculate the yield rate of each machine,
    # change NaN value with zeros
    machine_info['em_yieldrate'] = (machine_info['exp_num_good_pieces']     \
                                    / machine_info['sum_exp']               \
                                   ).replace(np.nan, 0)

    return machine_info

## Run EM Iterations

In [52]:
def em_algorithm(source_data, mach_info, lot_info):
    """
    Expectation–Maximization (EM) algorithm. Return most optimal or converged
    yieldrate.

    Parameter:
    ----------
    source_data : dataframe (2D array)
        full data from your csv, including the 'lot_step_machines' column.
    mach_info : dataframe (2D array)
        A machine information dataframe for each machine.

    Return:
    ----------
    machine_info : dataframe (2D array)
        An updated machine information dataframe for each machine. The column of
        'em_yieldrate' is already calculated by EM algorithm, and can be a
        benchmark for each machine's yield rate.

    """
    # initializing variables
    mach_info['em_yieldrate'] = mach_info['init_yieldrate']

    # record for EM iterations
    em_update = pd.DataFrame(data=None, columns=['last', 'current', 'diff'])

    #Start EM Iteration until Max Iteration
    for iters in range(MAX_EMA_ITERATIONS):
        # initializing
        # If some element of yieldrate <= 0 then replaces with a very small number
        mach_info['em_yieldrate'] = np.where(mach_info['em_yieldrate'] <= 0,
                                             EPSILON,
                                             mach_info['em_yieldrate'])
        mach_info['ln_yieldrate'] = np.log(mach_info['em_yieldrate'])
        mach_info['prev_yieldrate'] = mach_info['em_yieldrate']

        # E-STEP ###
        mach_info = e_step(source_data, mach_info)

        # M-STEP ###
        mach_info = m_step(mach_info)

        #show accuracy each iteration
        #_, accuracy, lot_bp_acc = em_accuracy(mach_info, lot_info)

        # Calculating Stop Condition
        tolerance = 1.e-3
        #it could save the update history for each iteration
        prev_yr = mach_info['prev_yieldrate'].values.copy()
        cur_yr = mach_info['em_yieldrate'].values.copy()
        diff = np.linalg.norm(prev_yr - cur_yr)

        #save to update record
        em_update.loc[len(em_update)] = [prev_yr, cur_yr, diff]

        """
        #show current progress / iteration
        cu.show_progress(iters+1, MAX_EMA_ITERATIONS,
                         messages="iteration: %d/%d;"
                         %(iters+1, MAX_EMA_ITERATIONS),
                         force_show=True)
        """
        #stop condition
        if diff <= tolerance:
            break

    #print()

    return mach_info, em_update, iters+1

# Report

In [53]:
def create_report(machine_info, target_filename):
    """
    Saving report to csv file. this will save only the yield rate and bad pieces
    per machine step.

    Parameter:
    ----------
    machine_statistic : DataFrame (2D array)
        a machine statistic dataframe which already processed by EM algorithm
        and have bad pieces information
    target_filename : str
        given path and filename such as ``"folder/yourfilename.csv"``

    """
    #sort by yieldrate from the smallest and get some report summary
    machine_info = machine_info.sort_values('em_yieldrate')
    report = machine_info[['em_yieldrate', 'exp_num_bad_pieces', 'machine_name']]

    #renaming columns to match to the dictionary documents
    report.columns = ['YieldRate', 'BadPiece', 'Machine']
    report = report.reset_index(drop=True)

    #changing display options to set print all column
    pd.set_option('display.max_columns', None)
    report = report.round(5) #cleaning some decimal
    #print(report)

    #saving report summaries
    report.to_csv(target_filename, sep=',', index=False)

    #check file
    check = pd.read_csv(target_filename, index_col=0)
    print("Saving %s Success" %(target_filename)
            if check.size == check.size \
            else "Saving Report %s failed" %(target_filename))

    #reset display options
    pd.reset_option('^display.', silent=True)

    return report

# Run EM Algorithm v1.2

In [54]:
def main(source_filename, report_filename):
    """
    Main procedure for running the EM Algorithm

    Parameter:
    ----------
    source_filename, report_filename : str
        given path and filename such as ``"folder/yourfilename.csv"``

    """
    start = time.time()
    print('Process Start: ', source_filename)

    # STEP 1: get raw data
    #print('Step 1: Reading CSV')
    raw_data = read_csv_files(source_filename)
    print(raw_data.head())


    #start_split = time.time()
    #print('Step 2: One Hot Encoding of Machine Steps')
    # STEP 2: create one hot encoding for each machine step
    # and add lot machine usage to raw_data
    onehot_lot_last_step, machine_information, raw_data = get_onehot_machine(raw_data)


    #start_split = time.time()
    #print('Step 3: Preprocesing: Lot Information')
    # STEP 3: calculate each LOT's statistics
    lot_information = get_lot_information(raw_data)


    #start_split = time.time()
    #print('Step 4: Preprocesing: Machine Information')
    # STEP 4: calculate each machine's statistics
    machine_information = get_machine_information(onehot_lot_last_step, machine_information, lot_information)


    #start_split = time.time()
    #print('Step 5: Running EM Algorithm')
    # STEP 5: calculate machine's yield rate using EM Algorithm
    machine_information, em_iter_res, iters = em_algorithm(raw_data, machine_information, lot_information)

    # STEP 6: creating reports
    #print('Step 6: Saving Report')
    report = create_report(machine_information, report_filename)

    #print the processing time
    #print('Process End')
    print('Process End: ', source_filename)
    cu.check_time(start, time.time())
    #print()

    return raw_data, onehot_lot_last_step, machine_information, lot_information, \
           machine_information, em_iter_res, iters, report

if __name__ == "__main__":
    raw_data, onehot_lot_last_step, machine_information, lot_information, \
    machine_information, em_iter_res, last_iters, report = main(RAWDATA_CSV_PATH_FILENAME, REPORT_PATH_FILENAME)

    print(report)

Process Start:  real_world_data.csv
              lot_name lot_machine_step001 lot_machine_step002  \
0  WB2018400438-C00001              FN-004                 NaN   
1  WB2018400438-C00001              FN-004              PT-106   
2  WB2018400438-C00001              FN-004              PT-106   
3  WB2018400438-C00001              FN-004              PT-106   
4  WB2018400438-C00001              FN-004              PT-106   

  lot_machine_step003 lot_machine_step004 lot_machine_step005  \
0                 NaN                 NaN                 NaN   
1                 NaN                 NaN                 NaN   
2              FN-007                 NaN                 NaN   
3              FN-007              PT-106                 NaN   
4              FN-007              PT-106              FN-010   

  lot_machine_step006 lot_machine_step007 lot_machine_step008  \
0                 NaN                 NaN                 NaN   
1                 NaN                 NaN     

<ipython-input-46-984773928a32>:31: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  onehot_machine_step = pd.get_dummies(machine_step_data,


Saving report_real_world_data.csv Success
Process End:  real_world_data.csv
Time elapsed: 0 H : 1 M : 5.463 Sec
     YieldRate     BadPiece Machine
0      0.04802     80.30365  LS-061
1      0.78655  21668.87927  PL-021
2      0.88132   6508.92055  SW-026
3      0.88493  13922.59752  LS-070
4      0.89059   2286.76895  GA-549
..         ...          ...     ...
211    1.00000      0.00000  FT-044
212    1.00000      0.00000  LS-068
213    1.00000      0.00000  AL-003
214    1.00000      0.00000  LS-064
215    1.00000      0.00000  CR-025

[216 rows x 3 columns]
